# 13. CNN 불량 이미지 검출 프로그램(2)
데이터셋의 불량 유형을 2가지로 설정   
VGG16 모델 적용

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import cv2
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.layers import Dense, Dropout, Input
from keras.models import Model, load_model

tf.compat.v1.disable_eager_execution()

In [4]:
import warnings
warnings.filterwarnings(action = 'ignore')

## Augmentation
데이터를 증폭하기 위한 기법  
* Rotation Range : 이미지 회전 각도 범위
* Width/Height Shift : 수평/수직 방향으로의 임의적인 평행이동의 범위
* Rescale : [0, 255] 범위의 RGB 값을 [0, 1] 범위로 정규화하는 전처리 과정
* Shear Range : 임의 전단 변환(직사각형 형태의 이미지를 평행사변형 형태로 변환) 범위
* Zoom Range : 확대 및 축소 범위
* Horizontal Flip : 이미지 수평 반전 허용 여부
* Fill Mode : 이미지를 조작할 때 발생하는 공백을 채우는 방식

In [7]:
datagen = ImageDataGenerator(
    rotation_range=3,
    width_shift_range=0.01,
    height_shift_range=0.01,
    rescale=1./255,
    shear_range=0.01,
    zoom_range=0.01,
    horizontal_flip=True,
    fill_mode='nearest')

for i in range(1, 21):
    path = 'CNN/Train/Broken/Broken' + str(i) + '.jpg'
    img = load_img(path)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)
    i = 0
    for batch in datagen.flow(x, batch_size=1, save_to_dir='CNN/Train/Broken', save_prefix='Broken', save_format='jpg'):
        i += 1
        if i > 50:
            break
            
for i in range(1, 21):
    path = 'CNN/Train/Circle/Circle' + str(i) + '.jpg'
    img = load_img(path)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)
    i = 0
    for batch in datagen.flow(x, batch_size=1, save_to_dir='CNN/Train/Circle', save_prefix='Circle', save_format='jpg'):
        i += 1
        if i > 50:
            break

for i in range(1, 21):
    path = 'CNN/Train/Original/Original' + str(i) + '.jpg'
    img = load_img(path)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)
    i = 0
    for batch in datagen.flow(x, batch_size=1, save_to_dir='CNN/Train/Original', save_prefix='Original', save_format='jpg'):
        i += 1
        if i > 50:
            break
            
print('이미지 생성 완료')

이미지 생성 완료


## VGG16 모델 불러오기
* VGG16은 16개의 층으로 이루어진 VGGNet(쉬운 구조와 좋은 성능으로 인해 널리 사용되는 CNN 네트워크)   
* 입력된 이미지 데이터셋을 1000개의 클래스로 분류하도록 학습된 모델
* 이 모델을 활용하여 원하는 개수의 클래스로 이미지를 분류하는 모델을 제작할 수 있음
* 224 X 224 사이즈로 이미지 인풋을 넣어주어야 하며, 커널 사이즈는 3 X 3

In [23]:
# 사전 학습된 모델 불러오기
input_tensor = Input(shape=(224,224,3))
model = VGG16(weights='imagenet', include_top=False, input_tensor = input_tensor)

## 모델 구성

In [22]:
# 모델 Layer 데이터화
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# Layer 추가
x = layer_dict['block5_pool'].output

# Cov2D Layer 추가
x = Conv2D(filters = 64, kernel_size=(3, 3), activation='relu')(x)

# MaxPooling2D Layer 추가
x = MaxPooling2D(pool_size=(2, 2))(x)

# Flatten Layer 추가
x = Flatten()(x)

# Fully Connected Layer 추가
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(3, activation='softmax')(x)

# 새 모델 정의
new_model = Model(inputs = model.input, outputs = x)

## 컴파일

In [34]:
# 사전에 학습된 가중치를 그대로 사용
for layer in new_model.layers[:19]:
    layer.trainable = False

new_model.summary()

# 컴파일 옵션
new_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

## 데이터 불러오기

In [35]:
train_dir = './CNN/Train'

# [0, 1] 범위로 정규화
train_image_generator = ImageDataGenerator(rescale=1./255)

# 데이터 구조 생성
train_data_gen = train_image_generator.flow_from_directory(batch_size=16, directory=train_dir,
                                                                   target_size=(224, 224), class_mode='binary')

Found 2954 images belonging to 3 classes.


## 학습

In [ ]:
# 모델 학습
history = new_model.fit(train_data_gen, epochs=3)

# 모델 저장
new_model.save('Model/newVGG16.h5')                        

# 최종 결과 리포트
acc = history.history['accuracy']
loss = history.history['loss']
epochs = range(len(acc))

Epoch 1/3
 61/185 [========>.....................] - ETA: 21:40 - batch: 30.0000 - size: 15.9016 - loss: 6.8499e-05 - accuracy: 1.0000

## 예측 결과 출력

In [25]:
# 저장 모델 불러오기
new_model = load_model("Model/Model.h5")

img = np.resize(cv2.imread('CNN/Test/Circle4.jpg', cv2.IMREAD_COLOR), (224, 224, 3))
inputData = []
inputData.append(img)
# print(new_model.predict(np.array(inputData)))

maximum = 0.0
maxIndex = 0
for array in new_model.predict(np.array(inputData)):
    for index, predict in enumerate(array):
        if maximum < float(predict):
            maximum = float(predict)
            maxIndex = index
            
if maxIndex == 0:
    print('결절형 결함 발생')
elif maxIndex == 1:
    print('원형 결함 발생')
else:
    print('정상')

[[8.455566e-14 9.999939e-01 6.105926e-06]]


## 예측 결과 전체 출력

In [33]:
import os

# 불러올 이미지들을 저장할 리스트
imgs = []

# 이미지들의 파일명들을 저장할 리스트
imageNames = []

# 이미지 불러오기
for imageName in os.listdir('./CNN/Test/'):
    # 불러올 이미지의 경로를 변수에 저장
    path = './CNN/Test/' + imageName
    
    # 이미지 불러오기
    img = np.resize(cv2.imread(path, cv2.IMREAD_COLOR), (224, 224, 3))
    imgs.append(img)

    # 불러온 이미지의 파일명을 리스트에 저장
    imageNames.append(imageName)

for i, result in enumerate(new_model.predict(np.array(imgs))):
    print(imageNames[i], end='\t')
    print(result)

.DS_Store	[nan nan nan]
Broken4.jpg	[1. 0. 0.]
Original.jpg	[1.3475853e-09 3.7665460e-19 1.0000000e+00]
Broken5.jpg	[1. 0. 0.]
Circle4.jpg	[8.455501e-14 9.999939e-01 6.105880e-06]
Circle.jpg	[1.3459718e-13 1.0000000e+00 0.0000000e+00]
Circle3.jpg	[2.0007302e-37 1.0000000e+00 0.0000000e+00]
Broken2.jpg	[1. 0. 0.]
Broken3.jpg	[1. 0. 0.]
Circle2.jpg	[9.280094e-21 1.000000e+00 0.000000e+00]
Broken.jpg	[1. 0. 0.]
